# Machine Learning For Lithography
## Unit III : Multilayer Perceptrons

### Introduction

The Multilayer Perceptron is a simple and useful neural network architecture.  It organizes the neurons into a sequence of "layers".  Each layer is composed of neurons. Each neuron in a layer is connected to every neuron in the preceding layer, with a connection strength determined by a learned weight.  Each neruon in layer maps the weighted sum of all of its inputs to an output activation, according to an activation function, offset by a scalar bias.

The multliayer perceptron may theoretically be considered a universald function approximator, as described in this paper: https://pdfs.semanticscholar.org/05ce/b32839c26c8d2cb38d5529cf7720a68c3fab.pdf

In practice, the difficulties of training MLPs with large numbers of layers and neurons inhibited further research into neural networks for many years, due to the tendency of the gradients used to calculate update directions to either vanish or explode.  Also, although the neutral form of the MLP makes it relatively bias-free, this can also make modeling more difficuilt if we have strong prior reasons to, for example, believe that the result of a classification should not be dependent on the placement of an image in the field, or if we believe think that the network of weights should be sparse.

Like the linear classifier, the network is trained with a  a dataset D consisting of training input data samples X and labels y(X).  

Also, the acceptable values of y are in a finite set of N different classifications C={c1, c2, ...cN}.

In the script we  will call the number of different classifications "n_classes."

The number of features per input sample X is 48x48 = 2304.



### Preamble
This section imports some necessary packages and helper functions that enable our script.

Of particular importance is TensorFlow, here imported as "tf," which is the nickname by which we will be able to access it in our script.  TensorFlow is our machine learning framework, enabling definition of the model form, definition of the training and validation procedures, definition of the model prediction method, and implementation of the training and prediction procedures.

We also import numpy, which we will reference with the nickname "np".  The name "numpy" is short for "numerical python".  The numpy package is a critical cornerstone of the data science workflow, providing intuitivce and interactive support for arrays in a fashion that will be familiar to those who have previously done work in matlab.

The matplotlib library is a nice set of tools for looking at our aerial images.

The methods loaded from "classes" are little helper functions I wrote to make the demo script you see more compact and focused on Machine Learning rather than munging data and logs and visualizations.

The preamble also sets some useful variables that help keep our log data separate from the other model forms.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from classes.Visualizations import *
from classes.Data import  loadResNIST
from classes.Specs import specs

import logging

DATADIR='./resNIST/'
LOGDIR = './perceptron_classifier_logs/'
PROJECTORDIR=LOGDIR+'projector/'
summary_writer = tf.summary.FileWriter(LOGDIR)

image_size=48
train_sample_count = 5000
eval_sample_count = 1000
#scopes = ['dnn/hiddenlayer_0/LeakyRelu:0','dnn/hiddenlayer_1/LeakyRelu:0','dnn/hiddenlayer_2/LeakyRelu:0','dnn/head/labels:0']
scopes = ['dnn/hiddenlayer_2/LeakyRelu:0','dnn/head/labels:0']

### Load and Transform Data

We introduce here the "tf.estimator" which is a facility from tensorflow that helps standardize the definition of model training, validation and predicition pipelines.

In this code block we are loading our data into four blocks:
1. **train_data** : the input training data **X**, representing a set of samples of aerial images, each 48x48 pixels. 
2. **train_labels** : the label **y(X)**, belonging to one of 11 classes, **c in C={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}**.  These class labels are integers, but they represent the proportion of the pixel vicinity that is "covered" by photoresist after development. A 0 denotes "not covered." A 1 denotes "fully or 100% covered."  Each increase in the index of the label correspondes to in increase in resist coverage of 10%.
3. **eval_data** : these samples **X** are held out from training so that we may evaluate the variance and detect potential overfitting.
4. **eval_labels** : these labels are sued in conjunction with **eval_data** to help detect overfitting.

The code then defines two input functions, one for training (**train_input_fn**) and one for evaluation (**eval_input_fn**), according to the "numpy_input_fn" spec which helps facilitate feeding tensorflow batches of samples.  We indicate that the training input function will be fed from the **train_data** and **train_labels** variables, and likewise the evaluation input function will be fed from the **eval_data** and **eval_labels** variables.

For training we specify a mini-batch size, which determines how how many samples are averaged together in determining an update direction for adjusting the weights.  

During training we shuffle the dataset before breaking it into mini-batches, to prevent correlations from data preparation from skewing results or avoid reliance on lucky fits.  However, to ensure consistency when evaluating the data during training we do not shuffle during evaluation.

Finally, we take the 48x48 image for each input sample and break it into a long 2304 row, with each pixel belonging to its own "feature column" for every image.  This feature_column will be the front end of our TensorFlow model.

In [ ]:
train_data, train_labels , eval_data, eval_labels    = loadResNIST(DATADIR)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=128,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
feature_columns=[tf.feature_column.numeric_column("x",shape=[image_size,image_size])]

### Define the Estimator
We use the RunConfig facility of the tf.estimator to hspecify how frequently we want to checkpoint the model (save intermediate results.). This also influences how frequently we will perform validation.

Because we are using a DNNClassifier canned estimator, it is very easy to define the model form.  It is actually very commonly the case that you can begin a machine learning project in tensorflow by adapting an existing model form, rather than writing it from scratch.  Here is another nice demo that also uses the DNNClassifier:
https://codeburst.io/use-tensorflow-dnnclassifier-estimator-to-classify-mnist-dataset-a7222bf9f940.

We are still defining 11 **n_classes** because we are binning resist coverage into tenths, including the terminal conditions of 0% coverage and 100% coverage.

The network architecture of an MLP is defined by a list of integers. The length of the listy is the number of hidden layers, and the value of each element is the number of neurons within each layer. In the default example provided, a 32-neuron first layer, 16-neuron second layer and 16-neuron 3rd layer are indicated by the list [32,16,16].

We offer an example of how to control the choice of the optimizer, which in this case is the simple "Gradient Descent Optimizer" that utilizes blame assignment on each neuron to estimate weight updates based on the magnitudes of mispredictions, recursively through each layer from the output to the input (backpropagation.). The learning rate is set to 1e-4 to prevent over-oscillation between minibatches, which would slow convergence.  Effectively this helps us average update directions for the weifghts over several batches.  The AdamOptimizer is also a common choice, and it provides a nice simple way to avoid tuning some of the gradient descent hyperparameters. However the complexity of Adam is beyond the scope of this course. You can experiment with how it works by modifying the script and replacing GradientDescentOptimizer with AdamOptimizer.

We also show how to select the activation function used by the neurons.  For many years, motivated possible by the biological analogy or arguments from probabilty, the sigmoid activation was used for neural networks. more recently, rectified linear units were found to be more practical for training because they did not saturate above threshold, and thereby the provided more gradient response during training.

I have opted to use the "leaky_relu" activation fucntion in this case.  I chose to do so because I noticed that, based on the Beholder visualization, gradient descent was not penetrating very deep in some of the MLP networks I was experimenting with.  A normal relu activation will have an input value of zero for all inputs below the threshold, and increase linearly after that.  This means there is no gradient felow threshold.  Usin a leaky relu means that there is still some small gradient even below threshold, which can bring some neurons back to life as the training proceeds.  

Despite the more lively looking Beholder charts however, I did not see a significantly better result using leaky_relu rahter than simple relu.

We also add the surprising "dropout" regularizer into the model, which during training will randomly "drop" connections between some of the neurons, making the network less likely to rely on a small number of connections to perform the classifications. It is truly surprising that dropout seems to help training, and the best argument I have heard about it is that it effectively trains multiple different network topologies at the samne time, and at inference it provides something like an ensemble of many similar-yet-not-identical networks. During validation and inference all of the neurons are always connected (no dropouts.)

In [ ]:
config=tf.estimator.RunConfig(save_checkpoints_steps=1000)

perceptronclassifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    config=config,
    hidden_units=[64, 32, 16],
    optimizer=tf.train.GradientDescentOptimizer(2e-5),
    n_classes=11,
    dropout=0.1,
    activation_fn=tf.nn.leaky_relu,
    model_dir=LOGDIR
)

We plan on using the "train_and_eval" method provided for the tf.estimator class, because it automates a periodic evaluation of the model during training, generating occasional checkpoints and then loading those checkpoints in to assess the model performance on the evaluation data.  In order to do this we need to activate the "logger" that logs data, and we need to define the **train_spec** and **eval_spec** which specify some of the details of the process, including directories for logged data, duration of the training process and frequency of logged data.

The function "specs" is a little helper function I wrote for added compactness of this lab exercise.

In [ ]:
logging.getLogger().setLevel(logging.INFO)  
# to show info about training progress in the terminal
train_spec, eval_spec = specs(train_input_fn, eval_input_fn, 
                              logdir=LOGDIR, projectordir=PROJECTORDIR, 
                              max_train_steps=500000, eval_steps = 200, 
                              scopes = scopes, name = 'mlp')

In this elegant line of code we ask tensorflow to begin the training process, with periodic evaluation, using the perceptronclassifier model and the training and eval specs we previously defined. Nice and compact!

In [ ]:
tf.estimator.train_and_evaluate(perceptronclassifier, train_spec, eval_spec)

This is how we can evaluate the accuracy of the model independent of the training data.

In [ ]:
accuracy_score = perceptronclassifier.evaluate(input_fn=eval_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

This code prepares the data and metadata for plotting in TensorBoard using the Principal Components Analysis (PCA) and t-SNE projection methods for visualizing in high-dimensions.  The prepare_projector and prepare_sprites functions are little tidy script I wrote to simplify the lab.

In [ ]:
prepare_projector(PROJECTORDIR, scopes)
prepare_sprites(PROJECTORDIR, eval_data)

We can see a specific example by picking an index and using the "matplotlib" library to make a nice picture.

In [ ]:
plt.imshow(eval_data[11])